# snort
Install [snort](https://www.snort.org) 

In [ ]:
!snort -c snort.conf -r modbus_test_data_part1.pcap -q -K none -A console | tee snort.out

# tcpdump

In [1]:
!tcpdump -D

1.wlp2s0 [Up, Running, Wireless]
2.any (Pseudo-device that captures on all interfaces) [Up, Running]
3.lo [Up, Running, Loopback]
4.bluetooth0 (Bluetooth adapter number 0) [Wireless, Association status unknown]
5.bluetooth-monitor (Bluetooth Linux Monitor) [Wireless]
6.nflog (Linux netfilter log (NFLOG) interface) [none]
7.nfqueue (Linux netfilter queue (NFQUEUE) interface) [none]
8.dbus-system (D-Bus system bus) [none]
9.dbus-session (D-Bus session bus) [none]


In [2]:
!tcpdump -i -n -v -c

tcpdump: option requires an argument -- 'c'
tcpdump version 4.99.0
libpcap version 1.10.0 (with TPACKET_V3)
OpenSSL 1.1.1l  24 Aug 2021
Usage: tcpdump [-AbdDefhHIJKlLnNOpqStuUvxX#] [ -B size ] [ -c count ] [--count]
		[ -C file_size ] [ -E algo:secret ] [ -F file ] [ -G seconds ]
		[ -i interface ] [ --immediate-mode ] [ -j tstamptype ]
		[ -M secret ] [ --number ] [ --print ] [ -Q in|out|inout ]
		[ -r file ] [ -s snaplen ] [ -T type ] [ --version ]
		[ -V file ] [ -w file ] [ -W filecount ] [ -y datalinktype ]
		[ --time-stamp-precision precision ] [ --micro ] [ --nano ]
		[ -z postrotate-command ] [ -Z user ] [ expression ]


In [3]:
### Write a pcap file from captured data
!tcpdump -w example.pcap

tcpdump: listening on wlp2s0, link-type EN10MB (Ethernet), snapshot length 262144 bytes
^C
110 packets captured
145 packets received by filter
0 packets dropped by kernel


In [ ]:
### Read a pcap file
!tcpdump -r run8.pcap

# PcapXray
Download here: [Github/PcapXray](https:github.com/Srinivas11789/PcapXray)

# tshark

In [ ]:
!tshark --autostop packets:20

#### remote unixoid machines
To capture traffic you may want to do so on a remote intermittent machine (router, switch, etc). If this is not dedicated monitoring hardware, like a managed switch or similar, you can `pcap` with tshark (fixed or continuous)
```bash
# manual / autocapture
tshark -w /tmp/ten-seconds.pcap --autostop duration:10
# write continuously, 
tshark -w /tmp/one-day.pcap --ring-buffer duration:86400
ls -lah | grep one-day.pcap
```

In [ ]:
!tshark -r run8.pcap -T json > run8.json

In [6]:
import json
global capture
with open('run8.json') as cap:
    capture = json.load(cap)

modbus_packets = [
    {'ip.src': packet['_source']['layers']['ip']['ip.src'],
     'ip.dst': packet['_source']['layers']['ip']['ip.dst'],
     'modbus':packet['_source']['layers']['modbus']}
    for packet in capture
    if 'modbus' in packet['_source']['layers']]

modbus_packets[0]

{'ip.src': '192.168.1.100',
 'ip.dst': '192.168.1.102',
 'modbus': {'modbus.func_code': '3',
  'modbus.reference_num': '8',
  'modbus.word_cnt': '4'}}

In [7]:
frequencies = {}
keystr = lambda k, p: ":".join([k, p[k]])
increment = lambda m, k: m[k] + 1 if k in m else 1
for packet in modbus_packets:
    frequencies[keystr('ip.src', packet)] = increment(frequencies, keystr('ip.src', packet))
    frequencies[keystr('ip.dst', packet)] = increment(frequencies, keystr('ip.dst', packet))
    frequencies[keystr('modbus.func_code', packet['modbus'])] = increment(frequencies, keystr('modbus.func_code', packet['modbus']))

dict(sorted(frequencies.items()))

{'ip.dst:192.168.1.100': 2212,
 'ip.dst:192.168.1.101': 1482,
 'ip.dst:192.168.1.102': 1473,
 'ip.dst:192.168.1.103': 1467,
 'ip.dst:192.168.1.99': 2209,
 'ip.src:192.168.1.100': 2213,
 'ip.src:192.168.1.101': 1482,
 'ip.src:192.168.1.102': 1472,
 'ip.src:192.168.1.103': 1467,
 'ip.src:192.168.1.99': 2209,
 'modbus.func_code:1': 2900,
 'modbus.func_code:2': 2902,
 'modbus.func_code:3': 2909,
 'modbus.func_code:5': 132}

# sharkd

In [ ]:
!echo '{"req": "info"}' | sharkd -

# Scapy

### Install

In [ ]:
!pip3 install scapy

## Basics

In [8]:
from scapy.all import IP

# create a basic IP-Packet
IP().show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = hopopt
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



### Monitoring
#### a simple network scanner
To monitor which devices are on your network, you may want to actively search for active IPs

In [ ]:
!ip address

In [ ]:
# Are you root?
# Make sure you have elevated privileges for the next step
!whoami

In [ ]:
# src: https://www.thepythoncode.com/article/building-networks-scanner-using-scapy
from scapy.all import ARP, Ether, srp
# set target IP range
target = "192.168.0.1/24" # your local subnet
# create ARP (Address Resolution protocol) packet
arp = ARP(pdst=target)
ether = Ether(dst="ff:ff:ff:ff:ff:ff")
packet = ether/arp

# raw socket operations require elevated privileges
result = srp(packet, timeout=10) # => Tuple (answered, unanswered)

print("===== Query Answer =====")
print(result[0][0])

print("===== Devices =====")
{received.psrc: received.hwsrc for sent, received in result[0]}

#### a simple network sniffer

In [ ]:
# adapted from src: https://www.thepythoncode.com/article/dhcp-listener-using-scapy-in-python
from scapy.all import sniff, Ether, DHCP

global print_packet, target_mac, requested_ip, hostname, vendor_id

def listen():
    sniff(prn=print_packet, iface='wlp2s0')

def print_packet(packet):
    print(packet.summary())
    #print(packet.show())

listen()

#### network "sniffer" - pcap style
It is easy to import a pcap file. 

In [ ]:
from scapy.all import rdpcap, PcapReader

fake_command_pcap = rdpcap('send_a_fake_command_modbus_6RTU_with_operate.pcap')
for packet in fake_command_pcap:
    print(packet.summary())

Using a contrib module can greatly benefit the output

In [ ]:
from scapy.contrib import modbus
fake_command_pcap_with_modbus = rdpcap('send_a_fake_command_modbus_6RTU_with_operate.pcap')
for packet in fake_command_pcap_with_modbus:
    print(packet.summary())